In [3]:
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, classification_report
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import os 
import pandas as pd
import numpy as np
import random
import pickle
import pandas as pd
from Bio import SeqIO
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import roc_curve

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import load_model
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, classification_report

In [4]:
os.chdir("/homes/t326h379/Ankh_nucleocytoplasmic_dbptm_2231")

df_positive = pd.read_csv("Nucleocytoplasmic_5724_with_1638_Proteins.csv")
df_negative = pd.read_csv("Nucleocytoplasmic_232286_with_1638_Proteins.csv")

df_positive_positive = df_positive
df_negative_negative = df_negative

Header_name = ["Unnamed: 0","Position","PID","S_or_T"]+[i for i in range(1,1025)]
df_positive_positive.columns = Header_name
df_negative_negative.columns = Header_name

In [6]:
df_positive.columns = Header_name
df_negative.columns = Header_name

In [5]:
df_positive_positive = df_positive_positive.drop('Unnamed: 0', axis=1)
df_negative_negative = df_negative_negative.drop('Unnamed: 0', axis=1)

In [40]:
a = random.sample(range(1, 1000000), 40)

for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride


    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")

        print("\n\n\n")

        model = Sequential()

        model.add(Input(shape=(1024,)))

        model.add(Dense(64,activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(2,activation='softmax'))



        model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])

        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                        monitor = 'val_accuracy',
                                        verbose=0, 
                                        save_weights_only=False,
                                        save_best_only=True)

        reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=5, verbose=1, min_delta=1e-4, mode='max')

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

        history = model.fit(X_train, y_train_1, epochs=10,verbose=1, batch_size=32)
        
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)
#         xval = X_val_full_ProtT5
#         yval = y_val_full_ProtT5
        print(xval.shape,yval.shape)
        
        Y_pred = model.predict(xval)
        Y_pred = (Y_pred > 0.5)
        y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
        y_pred = np.array(y_pred)

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("MLP Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(5165, 1024) (526, 1024)
(206954, 1024) (23866, 1024)
Training Dataset
(10330, 1024) (10330,)
Validation Dataset




Epoch 1/10
323/323 [==============================] - 1s 2ms/step - loss: 0.5706 - accuracy: 0.7066
Epoch 2/10
323/323 [==============================] - 1s 2ms/step - loss: 0.5229 - accuracy: 0.7407
Epoch 3/10
323/323 [==============================] - 1s 2ms/step - loss: 0.4999 - accuracy: 0.7586
Epoch 4/10
323/323 [==============================] - 0s 2ms/step - loss: 0.4777 - accuracy: 0.7735
Epoch 5/10
323/323 [==============================] - 1s 2ms/step - loss: 0.4557 - accuracy: 0.7865
Epoch 6/10
323/323 [==============================] - 0s 1ms/step - loss: 0.4284 - accuracy: 0.8067
Epoch 7/10
323/323 [==============================] - 0s 1ms/step - loss: 0.4050 - accuracy: 0.8174
Epoch 8/10
323/323 [==============================] - 0s 1ms/step - loss: 0.3808 - accuracy: 0.8348
Epoch 9/10
323/323 [==============================] - 0s 1ms/step - loss: 0.3563 - 

325/325 [==============================] - 1s 2ms/step - loss: 0.3437 - accuracy: 0.8526
(1052, 1024) (1052,)
33/33 [==============================] - 0s 770us/step
Independent Testing Matthews Correlation:  0.380258574385744
Confusion Matrix : 
 [[407 119]
 [210 316]]
Accuracy on test set:    0.6872623574144486
Sensitivity:    0.6007604562737643 	 Specificity:    0.7737642585551331
Precision  : 0.7264367816091954
Area Under Curve:    0.6872623574144487
(1052, 1024) (1052,)
(5271, 1024) (440, 1024)
(209764, 1024) (21473, 1024)
Training Dataset
(10542, 1024) (10542,)
Validation Dataset




Epoch 1/10
330/330 [==============================] - 1s 2ms/step - loss: 0.5684 - accuracy: 0.7041
Epoch 2/10
330/330 [==============================] - 1s 2ms/step - loss: 0.5232 - accuracy: 0.7432
Epoch 3/10
330/330 [==============================] - 1s 2ms/step - loss: 0.4974 - accuracy: 0.7618
Epoch 4/10
330/330 [==============================] - 1s 2ms/step - loss: 0.4716 - accuracy: 0.7789
Epoc

322/322 [==============================] - 1s 2ms/step - loss: 0.4626 - accuracy: 0.7821
Epoch 6/10
322/322 [==============================] - 1s 2ms/step - loss: 0.4396 - accuracy: 0.7989
Epoch 7/10
322/322 [==============================] - 0s 2ms/step - loss: 0.4160 - accuracy: 0.8151
Epoch 8/10
322/322 [==============================] - 1s 2ms/step - loss: 0.3918 - accuracy: 0.8272
Epoch 9/10
322/322 [==============================] - 1s 2ms/step - loss: 0.3686 - accuracy: 0.8409
Epoch 10/10
322/322 [==============================] - 0s 1ms/step - loss: 0.3449 - accuracy: 0.8572
(1122, 1024) (1122,)
36/36 [==============================] - 0s 994us/step
Independent Testing Matthews Correlation:  0.4606907816821446
Confusion Matrix : 
 [[426 135]
 [168 393]]
Accuracy on test set:    0.7299465240641712
Sensitivity:    0.7005347593582888 	 Specificity:    0.7593582887700535
Precision  : 0.7443181818181818
Area Under Curve:    0.7299465240641712
(1122, 1024) (1122,)
(5102, 1024) (609, 

Epoch 1/10
319/319 [==============================] - 1s 2ms/step - loss: 0.5764 - accuracy: 0.7026
Epoch 2/10
319/319 [==============================] - 0s 1ms/step - loss: 0.5322 - accuracy: 0.7330
Epoch 3/10
319/319 [==============================] - 1s 2ms/step - loss: 0.5073 - accuracy: 0.7519
Epoch 4/10
319/319 [==============================] - 1s 2ms/step - loss: 0.4862 - accuracy: 0.7704
Epoch 5/10
319/319 [==============================] - 0s 2ms/step - loss: 0.4653 - accuracy: 0.7794
Epoch 6/10
319/319 [==============================] - 0s 1ms/step - loss: 0.4380 - accuracy: 0.7989
Epoch 7/10
319/319 [==============================] - 1s 2ms/step - loss: 0.4210 - accuracy: 0.8055
Epoch 8/10
319/319 [==============================] - 1s 2ms/step - loss: 0.3968 - accuracy: 0.8270
Epoch 9/10
319/319 [==============================] - 1s 2ms/step - loss: 0.3725 - accuracy: 0.8438
Epoch 10/10
319/319 [==============================] - 1s 2ms/step - loss: 0.3508 - accuracy: 0.8532

Independent Testing Matthews Correlation:  0.4554941263322754
Confusion Matrix : 
 [[489 185]
 [182 492]]
Accuracy on test set:    0.7277448071216617
Sensitivity:    0.7299703264094956 	 Specificity:    0.7255192878338279
Precision  : 0.7267355982274741
Area Under Curve:    0.7277448071216618
(1348, 1024) (1348,)
(4912, 1024) (707, 1024)
(206259, 1024) (24571, 1024)
Training Dataset
(9824, 1024) (9824,)
Validation Dataset




Epoch 1/10
307/307 [==============================] - 1s 2ms/step - loss: 0.5867 - accuracy: 0.6870
Epoch 2/10
307/307 [==============================] - 1s 2ms/step - loss: 0.5415 - accuracy: 0.7297
Epoch 3/10
307/307 [==============================] - 0s 1ms/step - loss: 0.5191 - accuracy: 0.7431
Epoch 4/10
307/307 [==============================] - 0s 1ms/step - loss: 0.4969 - accuracy: 0.7606
Epoch 5/10
307/307 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.7767
Epoch 6/10
307/307 [==============================] - 0s 2ms/step - lo

...

(5242, 1024) (425, 1024)
(210506, 1024) (19560, 1024)
Training Dataset
(10484, 1024) (10484,)
Validation Dataset




Epoch 1/10
328/328 [==============================] - 1s 1ms/step - loss: 0.5694 - accuracy: 0.7049
Epoch 2/10
328/328 [==============================] - 0s 1ms/step - loss: 0.5205 - accuracy: 0.7414
Epoch 3/10
328/328 [==============================] - 0s 1ms/step - loss: 0.4966 - accuracy: 0.7603
Epoch 4/10
328/328 [==============================] - 0s 1ms/step - loss: 0.4717 - accuracy: 0.7799
Epoch 5/10
328/328 [==============================] - 0s 1ms/step - loss: 0.4511 - accuracy: 0.7886
Epoch 6/10
328/328 [==============================] - 0s 1ms/step - loss: 0.4273 - accuracy: 0.8060
Epoch 7/10
328/328 [==============================] - 0s 1ms/step - loss: 0.4022 - accuracy: 0.8193
Epoch 8/10
328/328 [==============================] - 0s 1ms/step - loss: 0.3785 - accuracy: 0.8332
Epoch 9/10
328/328 [==============================] - 0s 1ms/step - loss: 0.3591 - 

322/322 [==============================] - 0s 1ms/step - loss: 0.3343 - accuracy: 0.8565
(1128, 1024) (1128,)
36/36 [==============================] - 0s 645us/step
Independent Testing Matthews Correlation:  0.36417708374198815
Confusion Matrix : 
 [[402 162]
 [197 367]]
Accuracy on test set:    0.6817375886524822
Sensitivity:    0.650709219858156 	 Specificity:    0.7127659574468085
Precision  : 0.6937618147448015
Area Under Curve:    0.6817375886524822
(1128, 1024) (1128,)
(5205, 1024) (500, 1024)
(209256, 1024) (21466, 1024)
Training Dataset
(10410, 1024) (10410,)
Validation Dataset




Epoch 1/10
326/326 [==============================] - 1s 1ms/step - loss: 0.5721 - accuracy: 0.7031
Epoch 2/10
326/326 [==============================] - 0s 1ms/step - loss: 0.5259 - accuracy: 0.7415
Epoch 3/10
326/326 [==============================] - 0s 1ms/step - loss: 0.5021 - accuracy: 0.7553
Epoch 4/10
326/326 [==============================] - 0s 1ms/step - loss: 0.4774 - accuracy: 0.7720
Epo

Training Dataset
(10002, 1024) (10002,)
Validation Dataset




Epoch 1/10
313/313 [==============================] - 1s 1ms/step - loss: 0.5828 - accuracy: 0.6993
Epoch 2/10
313/313 [==============================] - 0s 1ms/step - loss: 0.5361 - accuracy: 0.7344
Epoch 3/10
313/313 [==============================] - 0s 1ms/step - loss: 0.5151 - accuracy: 0.7529
Epoch 4/10
313/313 [==============================] - 0s 1ms/step - loss: 0.4909 - accuracy: 0.7709
Epoch 5/10
313/313 [==============================] - 0s 1ms/step - loss: 0.4677 - accuracy: 0.7800
Epoch 6/10
313/313 [==============================] - 0s 1ms/step - loss: 0.4445 - accuracy: 0.7983
Epoch 7/10
313/313 [==============================] - 0s 1ms/step - loss: 0.4219 - accuracy: 0.8110
Epoch 8/10
313/313 [==============================] - 0s 1ms/step - loss: 0.3954 - accuracy: 0.8266
Epoch 9/10
313/313 [==============================] - 0s 1ms/step - loss: 0.3731 - accuracy: 0.8405
Epoch 10/10
313/313 [================

328/328 [==============================] - 0s 1ms/step - loss: 0.3362 - accuracy: 0.8579
(946, 1024) (946,)
30/30 [==============================] - 0s 659us/step
Independent Testing Matthews Correlation:  0.3944433254183425
Confusion Matrix : 
 [[348 125]
 [162 311]]
Accuracy on test set:    0.6966173361522199
Sensitivity:    0.6575052854122622 	 Specificity:    0.7357293868921776
Precision  : 0.713302752293578
Area Under Curve:    0.6966173361522199
(946, 1024) (946,)
(5184, 1024) (529, 1024)
(208925, 1024) (22760, 1024)
Training Dataset
(10368, 1024) (10368,)
Validation Dataset




Epoch 1/10
324/324 [==============================] - 1s 1ms/step - loss: 0.5763 - accuracy: 0.7014
Epoch 2/10
324/324 [==============================] - 0s 1ms/step - loss: 0.5323 - accuracy: 0.7342
Epoch 3/10
324/324 [==============================] - 0s 1ms/step - loss: 0.5056 - accuracy: 0.7541
Epoch 4/10
324/324 [==============================] - 0s 1ms/step - loss: 0.4815 - accuracy: 0.7683
Epoch 5/

(5095, 1024) (614, 1024)
(206645, 1024) (24673, 1024)
Training Dataset
(10190, 1024) (10190,)
Validation Dataset




Epoch 1/10
319/319 [==============================] - 1s 1ms/step - loss: 0.5724 - accuracy: 0.7057
Epoch 2/10
319/319 [==============================] - 0s 1ms/step - loss: 0.5274 - accuracy: 0.7365
Epoch 3/10
319/319 [==============================] - 0s 1ms/step - loss: 0.5033 - accuracy: 0.7538
Epoch 4/10
319/319 [==============================] - 0s 1ms/step - loss: 0.4798 - accuracy: 0.7693
Epoch 5/10
319/319 [==============================] - 0s 1ms/step - loss: 0.4607 - accuracy: 0.7876
Epoch 6/10
319/319 [==============================] - 0s 1ms/step - loss: 0.4338 - accuracy: 0.8033
Epoch 7/10
319/319 [==============================] - 0s 1ms/step - loss: 0.4100 - accuracy: 0.8137
Epoch 8/10
319/319 [==============================] - 0s 1ms/step - loss: 0.3879 - accuracy: 0.8286
Epoch 9/10
319/319 [==============================] - 0s 1ms/step - loss: 0.3650 - 

# Logistic Regression 10 Fold CV

In [6]:
a = random.sample(range(1, 1000000), 10)
from sklearn.linear_model import LogisticRegression

for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride


    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")

        print("\n\n\n")

        clf = LogisticRegression(penalty='l2',solver='lbfgs',random_state=seed)

        clf.fit(X_train, y_train)
        
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)

        print(xval.shape,yval.shape)   
        
        y_pred = np.array(clf.predict(xval))

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("LR Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(4978, 1024) (731, 1024)
(207412, 1024) (23957, 1024)
Training Dataset
(9956, 1024) (9956,)
Validation Dataset






/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(1462, 1024) (1462,)
Independent Testing Matthews Correlation:  0.45237635660917486
Confusion Matrix : 
 [[483 248]
 [155 576]]
Accuracy on test set:    0.7243502051983585
Sensitivity:    0.7879616963064295 	 Specificity:    0.6607387140902873
Precision  : 0.6990291262135923
Area Under Curve:    0.7243502051983584
(1462, 1024) (1462,)
(5027, 1024) (682, 1024)
(203698, 1024) (27671, 1024)
Training Dataset
(10054, 1024) (10054,)
Validation Dataset






/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(1364, 1024) (1364,)
Independent Testing Matthews Correlation:  0.4254413524242996
Confusion Matrix : 
 [[497 185]
 [207 475]]
Accuracy on test set:    0.7126099706744868
Sensitivity:    0.6964809384164223 	 Specificity:    0.7287390029325513
Precision  : 0.7196969696969697
Area Under Curve:    0.7126099706744868
(1364, 1024) (1364,)
(5210, 1024) (499, 1024)
(208490, 1024) (22879, 1024)
Training Dataset
(10420, 1024) (10420,)
Validation Dataset






...

/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(1124, 1024) (1124,)
Independent Testing Matthews Correlation:  0.4395024751187659
Confusion Matrix : 
 [[405 157]
 [158 404]]
Accuracy on test set:    0.7197508896797153
Sensitivity:    0.7188612099644128 	 Specificity:    0.7206405693950177
Precision  : 0.7201426024955436
Area Under Curve:    0.7197508896797153
(1124, 1024) (1124,)
(5243, 1024) (466, 1024)
(209805, 1024) (21474, 1024)
Training Dataset
(10486, 1024) (10486,)
Validation Dataset






/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(932, 1024) (932,)
Independent Testing Matthews Correlation:  0.3688488360717199
Confusion Matrix : 
 [[360 106]
 [191 275]]
Accuracy on test set:    0.6813304721030042
Sensitivity:    0.5901287553648069 	 Specificity:    0.7725321888412017
Precision  : 0.7217847769028871
Area Under Curve:    0.6813304721030045
(932, 1024) (932,)
(5293, 1024) (416, 1024)
(208783, 1024) (22496, 1024)
Training Dataset
(10586, 1024) (10586,)
Validation Dataset




(832, 1024) (832,)
Independent Testing Matthews Correlation:  0.3874085329801385
Confusion Matrix : 
 [[328  88]
 [170 246]]
Accuracy on test set:    0.6899038461538461
Sensitivity:    0.5913461538461539 	 Specificity:    0.7884615384615384
Precision  : 0.7365269461077845
Area Under Curve:    0.6899038461538461
(832, 1024) (832,)
LR Mean MCC = 0.3951 + 0.0447 and Mean AUC = 0.6963 + 0.0228
Mean Sensitivity = 0.6560 + 0.0568 and Mean Specificity = 0.7367 + 0.0315 and Mean Accuracy = 0.6963 + 0.0228
Mean Precision = 0.7139 + 0.0201







/homes/t326h379/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Random Forest 10 Fold CV

In [5]:
a = random.sample(range(1, 1000000), 10)
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride


    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")

        print("\n\n\n")

        clf = RandomForestClassifier(max_depth=10, random_state=seed)

        clf.fit(X_train, y_train)
        
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)

        print(xval.shape,yval.shape)
        
        y_pred = np.array(clf.predict(xval))

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("RF Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(5227, 1024) (488, 1024)
(207825, 1024) (23627, 1024)
Training Dataset
(10454, 1024) (10454,)
Validation Dataset




(976, 1024) (976,)
Independent Testing Matthews Correlation:  0.4084336179779369
Confusion Matrix : 
 [[392  96]
 [197 291]]
Accuracy on test set:    0.6997950819672131
Sensitivity:    0.5963114754098361 	 Specificity:    0.8032786885245902
Precision  : 0.751937984496124
Area Under Curve:    0.6997950819672131
(976, 1024) (976,)
(5151, 1024) (564, 1024)
(207823, 1024) (23629, 1024)
Training Dataset
(10302, 1024) (10302,)
Validation Dataset




(1128, 1024) (1128,)
Independent Testing Matthews Correlation:  0.30191616247032926
Confusion Matrix : 
 [[412 152]
 [244 320]]
Accuracy on test set:    0.648936170212766
Sensitivity:    0.5673758865248227 	 Specificity:    0.7304964539007093
Precision  : 0.6779661016949152
Area Under Curve:    0.648936170212766
(1128, 1024) (1128,)
(5126, 1024) (589, 1024)
(207350, 1024) (24102, 1024)
Training Dataset
(10252, 1024) (10252,)
Valida

(5042, 1024) (643, 1024)
(205324, 1024) (24893, 1024)
Training Dataset
(10084, 1024) (10084,)
Validation Dataset




(1286, 1024) (1286,)
Independent Testing Matthews Correlation:  0.3593071190955099
Confusion Matrix : 
 [[507 136]
 [282 361]]
Accuracy on test set:    0.6749611197511665
Sensitivity:    0.5614307931570762 	 Specificity:    0.7884914463452566
Precision  : 0.7263581488933601
Area Under Curve:    0.6749611197511663
(1286, 1024) (1286,)
(5193, 1024) (492, 1024)
(207338, 1024) (22879, 1024)
Training Dataset
(10386, 1024) (10386,)
Validation Dataset




(984, 1024) (984,)
Independent Testing Matthews Correlation:  0.30148226782909887
Confusion Matrix : 
 [[377 115]
 [233 259]]
Accuracy on test set:    0.6463414634146342
Sensitivity:    0.5264227642276422 	 Specificity:    0.766260162601626
Precision  : 0.6925133689839572
Area Under Curve:    0.646341463414634
(984, 1024) (984,)
(5050, 1024) (635, 1024)
(204156, 1024) (26061, 1024)
Training Dataset
(10100, 1024) (10100,)
Valid

(5191, 1024) (465, 1024)
(210148, 1024) (20851, 1024)
Training Dataset
(10382, 1024) (10382,)
Validation Dataset




(930, 1024) (930,)
Independent Testing Matthews Correlation:  0.3165103663427937
Confusion Matrix : 
 [[359 106]
 [216 249]]
Accuracy on test set:    0.6537634408602151
Sensitivity:    0.535483870967742 	 Specificity:    0.7720430107526882
Precision  : 0.7014084507042253
Area Under Curve:    0.6537634408602151
(930, 1024) (930,)
(4899, 1024) (757, 1024)
(205605, 1024) (25394, 1024)
Training Dataset
(9798, 1024) (9798,)
Validation Dataset




(1514, 1024) (1514,)
Independent Testing Matthews Correlation:  0.555247492118802
Confusion Matrix : 
 [[618 139]
 [199 558]]
Accuracy on test set:    0.7767503302509907
Sensitivity:    0.7371202113606341 	 Specificity:    0.8163804491413474
Precision  : 0.8005738880918221
Area Under Curve:    0.7767503302509907
(1514, 1024) (1514,)
(5242, 1024) (414, 1024)
(206236, 1024) (24763, 1024)
Training Dataset
(10484, 1024) (10484,)
Validati

(5035, 1024) (652, 1024)
(205886, 1024) (25056, 1024)
Training Dataset
(10070, 1024) (10070,)
Validation Dataset




(1304, 1024) (1304,)
Independent Testing Matthews Correlation:  0.42192370844305355
Confusion Matrix : 
 [[546 106]
 [281 371]]
Accuracy on test set:    0.7032208588957055
Sensitivity:    0.5690184049079755 	 Specificity:    0.8374233128834356
Precision  : 0.7777777777777778
Area Under Curve:    0.7032208588957055
(1304, 1024) (1304,)
(5164, 1024) (523, 1024)
(207448, 1024) (23494, 1024)
Training Dataset
(10328, 1024) (10328,)
Validation Dataset




(1046, 1024) (1046,)
Independent Testing Matthews Correlation:  0.4105448277425146
Confusion Matrix : 
 [[421 102]
 [211 312]]
Accuracy on test set:    0.7007648183556405
Sensitivity:    0.5965583173996176 	 Specificity:    0.8049713193116634
Precision  : 0.7536231884057971
Area Under Curve:    0.7007648183556405
(1046, 1024) (1046,)
(5187, 1024) (500, 1024)
(206558, 1024) (24384, 1024)
Training Dataset
(10374, 1024) (10374,)

(5254, 1024) (451, 1024)
(206188, 1024) (23830, 1024)
Training Dataset
(10508, 1024) (10508,)
Validation Dataset




(902, 1024) (902,)
Independent Testing Matthews Correlation:  0.2754484336160164
Confusion Matrix : 
 [[355  96]
 [237 214]]
Accuracy on test set:    0.6308203991130821
Sensitivity:    0.4745011086474501 	 Specificity:    0.7871396895787139
Precision  : 0.6903225806451613
Area Under Curve:    0.6308203991130821
(902, 1024) (902,)
(5005, 1024) (700, 1024)
(206946, 1024) (23072, 1024)
Training Dataset
(10010, 1024) (10010,)
Validation Dataset




(1400, 1024) (1400,)
Independent Testing Matthews Correlation:  0.43244580668396315
Confusion Matrix : 
 [[576 124]
 [281 419]]
Accuracy on test set:    0.7107142857142857
Sensitivity:    0.5985714285714285 	 Specificity:    0.8228571428571428
Precision  : 0.7716390423572744
Area Under Curve:    0.7107142857142857
(1400, 1024) (1400,)
(4936, 1024) (769, 1024)
(206567, 1024) (23451, 1024)
Training Dataset
(9872, 1024) (9872,)
Valid

# XGBoost 10 Fold CV

In [7]:
a = random.sample(range(1, 1000000), 10)
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride


    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")

        print("\n\n\n")

        clf = XGBClassifier()

        clf.fit(X_train, y_train)
        
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)

        print(xval.shape,yval.shape)
        
        y_pred = np.array(clf.predict(xval))

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("XGBoost Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(5240, 1024) (475, 1024)
(207219, 1024) (24327, 1024)
Training Dataset
(10480, 1024) (10480,)
Validation Dataset




(950, 1024) (950,)
Independent Testing Matthews Correlation:  0.3574869889149701
Confusion Matrix : 
 [[364 111]
 [197 278]]
Accuracy on test set:    0.6757894736842105
Sensitivity:    0.5852631578947368 	 Specificity:    0.7663157894736842
Precision  : 0.7146529562982005
Area Under Curve:    0.6757894736842105
(950, 1024) (950,)
(5244, 1024) (471, 1024)
(208323, 1024) (23223, 1024)
Training Dataset
(10488, 1024) (10488,)
Validation Dataset




(942, 1024) (942,)
Independent Testing Matthews Correlation:  0.34636473260027856
Confusion Matrix : 
 [[354 117]
 [193 278]]
Accuracy on test set:    0.6709129511677282
Sensitivity:    0.5902335456475584 	 Specificity:    0.7515923566878981
Precision  : 0.7037974683544304
Area Under Curve:    0.6709129511677283
(942, 1024) (942,)
(5222, 1024) (493, 1024)
(205328, 1024) (26218, 1024)
Training Dataset
(10444, 1024) (10444,)
Validat

(5190, 1024) (513, 1024)
(207581, 1024) (23835, 1024)
Training Dataset
(10380, 1024) (10380,)
Validation Dataset




(1026, 1024) (1026,)
Independent Testing Matthews Correlation:  0.38451632232442445
Confusion Matrix : 
 [[406 107]
 [213 300]]
Accuracy on test set:    0.6881091617933723
Sensitivity:    0.5847953216374269 	 Specificity:    0.7914230019493177
Precision  : 0.7371007371007371
Area Under Curve:    0.6881091617933723
(1026, 1024) (1026,)
(4967, 1024) (736, 1024)
(210238, 1024) (21178, 1024)
Training Dataset
(9934, 1024) (9934,)
Validation Dataset




(1472, 1024) (1472,)
Independent Testing Matthews Correlation:  0.46740683605349975
Confusion Matrix : 
 [[537 199]
 [193 543]]
Accuracy on test set:    0.7336956521739131
Sensitivity:    0.7377717391304348 	 Specificity:    0.7296195652173914
Precision  : 0.7318059299191375
Area Under Curve:    0.7336956521739131
(1472, 1024) (1472,)
(5102, 1024) (601, 1024)
(209019, 1024) (22397, 1024)
Training Dataset
(10204, 1024) (10204,)


(5030, 1024) (663, 1024)
(211224, 1024) (20080, 1024)
Training Dataset
(10060, 1024) (10060,)
Validation Dataset




(1326, 1024) (1326,)
Independent Testing Matthews Correlation:  0.36282764950005
Confusion Matrix : 
 [[474 189]
 [234 429]]
Accuracy on test set:    0.6809954751131222
Sensitivity:    0.6470588235294118 	 Specificity:    0.7149321266968326
Precision  : 0.6941747572815534
Area Under Curve:    0.6809954751131222
(1326, 1024) (1326,)
(5197, 1024) (496, 1024)
(207562, 1024) (23742, 1024)
Training Dataset
(10394, 1024) (10394,)
Validation Dataset




(992, 1024) (992,)
Independent Testing Matthews Correlation:  0.3329079492933485
Confusion Matrix : 
 [[377 119]
 [215 281]]
Accuracy on test set:    0.6633064516129032
Sensitivity:    0.5665322580645161 	 Specificity:    0.7600806451612904
Precision  : 0.7025
Area Under Curve:    0.6633064516129032
(992, 1024) (992,)
(5232, 1024) (461, 1024)
(205527, 1024) (25777, 1024)
Training Dataset
(10464, 1024) (10464,)
Validation Dataset

(5057, 1024) (624, 1024)
(207801, 1024) (23504, 1024)
Training Dataset
(10114, 1024) (10114,)
Validation Dataset




(1248, 1024) (1248,)
Independent Testing Matthews Correlation:  0.39324728206651227
Confusion Matrix : 
 [[452 172]
 [207 417]]
Accuracy on test set:    0.6963141025641025
Sensitivity:    0.6682692307692307 	 Specificity:    0.7243589743589743
Precision  : 0.7079796264855688
Area Under Curve:    0.6963141025641025
(1248, 1024) (1248,)
(5031, 1024) (650, 1024)
(206650, 1024) (24655, 1024)
Training Dataset
(10062, 1024) (10062,)
Validation Dataset




(1300, 1024) (1300,)
Independent Testing Matthews Correlation:  0.4454178596414401
Confusion Matrix : 
 [[489 161]
 [200 450]]
Accuracy on test set:    0.7223076923076923
Sensitivity:    0.6923076923076923 	 Specificity:    0.7523076923076923
Precision  : 0.7364975450081833
Area Under Curve:    0.7223076923076923
(1300, 1024) (1300,)
(5177, 1024) (504, 1024)
(208790, 1024) (22515, 1024)
Training Dataset
(10354, 1024) (10354,)

(4842, 1024) (855, 1024)
(208098, 1024) (23560, 1024)
Training Dataset
(9684, 1024) (9684,)
Validation Dataset




(1710, 1024) (1710,)
Independent Testing Matthews Correlation:  0.46199335996320684
Confusion Matrix : 
 [[627 228]
 [232 623]]
Accuracy on test set:    0.7309941520467836
Sensitivity:    0.7286549707602339 	 Specificity:    0.7333333333333333
Precision  : 0.7320799059929495
Area Under Curve:    0.7309941520467836
(1710, 1024) (1710,)
(5176, 1024) (521, 1024)
(209571, 1024) (22087, 1024)
Training Dataset
(10352, 1024) (10352,)
Validation Dataset




(1042, 1024) (1042,)
Independent Testing Matthews Correlation:  0.40440967514758785
Confusion Matrix : 
 [[398 123]
 [189 332]]
Accuracy on test set:    0.7005758157389635
Sensitivity:    0.6372360844529751 	 Specificity:    0.763915547024952
Precision  : 0.7296703296703296
Area Under Curve:    0.7005758157389634
(1042, 1024) (1042,)
(5136, 1024) (561, 1024)
(210426, 1024) (21232, 1024)
Training Dataset
(10272, 1024) (10272,)
V

# SVM 10 Fold CV

In [8]:
a = random.sample(range(1, 1000000), 10)
from sklearn import svm
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride


    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_ProtT5 = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_ProtT5 = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_ProtT5 = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_ProtT5 = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_ProtT5,y_train_full_ProtT5)
        print(X_train.shape,y_train.shape)

        y_train_1 = tf.keras.utils.to_categorical(y_train,2)

        print("Validation Dataset")

        print("\n\n\n")

        clf = svm.SVC(C=10, kernel='rbf')

        clf.fit(X_train, y_train)
        
        xval,yval = rus.fit_resample(X_val_full_ProtT5,y_val_full_ProtT5)

        print(xval.shape,yval.shape)
        
        y_pred = np.array(clf.predict(xval))

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("SVM Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(5276, 1024) (427, 1024)
(205573, 1024) (24614, 1024)
Training Dataset
(10552, 1024) (10552,)
Validation Dataset




(854, 1024) (854,)
Independent Testing Matthews Correlation:  0.3909350184867625
Confusion Matrix : 
 [[346  81]
 [184 243]]
Accuracy on test set:    0.689695550351288
Sensitivity:    0.5690866510538641 	 Specificity:    0.810304449648712
Precision  : 0.75
Area Under Curve:    0.689695550351288
(854, 1024) (854,)
(5207, 1024) (496, 1024)
(208166, 1024) (22021, 1024)
Training Dataset
(10414, 1024) (10414,)
Validation Dataset




(992, 1024) (992,)
Independent Testing Matthews Correlation:  0.3799000760313548
Confusion Matrix : 
 [[372 124]
 [185 311]]
Accuracy on test set:    0.688508064516129
Sensitivity:    0.6270161290322581 	 Specificity:    0.75
Precision  : 0.7149425287356321
Area Under Curve:    0.688508064516129
(992, 1024) (992,)
(4899, 1024) (804, 1024)
(206320, 1024) (23867, 1024)
Training Dataset
(9798, 1024) (9798,)
Validation Dataset




(1608, 1024) (1608,)

(5167, 1024) (534, 1024)
(207069, 1024) (23502, 1024)
Training Dataset
(10334, 1024) (10334,)
Validation Dataset




(1068, 1024) (1068,)
Independent Testing Matthews Correlation:  0.3434573670238607
Confusion Matrix : 
 [[391 143]
 [209 325]]
Accuracy on test set:    0.6704119850187266
Sensitivity:    0.6086142322097379 	 Specificity:    0.7322097378277154
Precision  : 0.6944444444444444
Area Under Curve:    0.6704119850187266
(1068, 1024) (1068,)
(5239, 1024) (462, 1024)
(209150, 1024) (21421, 1024)
Training Dataset
(10478, 1024) (10478,)
Validation Dataset




(924, 1024) (924,)
Independent Testing Matthews Correlation:  0.26833620750695486
Confusion Matrix : 
 [[349 113]
 [229 233]]
Accuracy on test set:    0.6298701298701299
Sensitivity:    0.5043290043290043 	 Specificity:    0.7554112554112554
Precision  : 0.6734104046242775
Area Under Curve:    0.6298701298701299
(924, 1024) (924,)
(5044, 1024) (657, 1024)
(205558, 1024) (25013, 1024)
Training Dataset
(10088, 1024) (10088,)
Val

(4905, 1024) (778, 1024)
(209929, 1024) (21070, 1024)
Training Dataset
(9810, 1024) (9810,)
Validation Dataset




(1556, 1024) (1556,)
Independent Testing Matthews Correlation:  0.4935879433705334
Confusion Matrix : 
 [[584 194]
 [200 578]]
Accuracy on test set:    0.7467866323907455
Sensitivity:    0.7429305912596401 	 Specificity:    0.7506426735218509
Precision  : 0.7487046632124352
Area Under Curve:    0.7467866323907455
(1556, 1024) (1556,)
(5091, 1024) (592, 1024)
(207098, 1024) (23901, 1024)
Training Dataset
(10182, 1024) (10182,)
Validation Dataset




(1184, 1024) (1184,)
Independent Testing Matthews Correlation:  0.3539121883631855
Confusion Matrix : 
 [[453 139]
 [247 345]]
Accuracy on test set:    0.6739864864864865
Sensitivity:    0.5827702702702703 	 Specificity:    0.7652027027027027
Precision  : 0.7128099173553719
Area Under Curve:    0.6739864864864864
(1184, 1024) (1184,)
(5158, 1024) (525, 1024)
(207818, 1024) (23181, 1024)
Training Dataset
(10316, 1024) (10316,)
Va

(4969, 1024) (745, 1024)
(208914, 1024) (22193, 1024)
Training Dataset
(9938, 1024) (9938,)
Validation Dataset




(1490, 1024) (1490,)
Independent Testing Matthews Correlation:  0.4086139617258471
Confusion Matrix : 
 [[544 201]
 [240 505]]
Accuracy on test set:    0.7040268456375839
Sensitivity:    0.6778523489932886 	 Specificity:    0.7302013422818792
Precision  : 0.7152974504249292
Area Under Curve:    0.7040268456375839
(1490, 1024) (1490,)
(5158, 1024) (556, 1024)
(205651, 1024) (25456, 1024)
Training Dataset
(10316, 1024) (10316,)
Validation Dataset




(1112, 1024) (1112,)
Independent Testing Matthews Correlation:  0.3617733366633708
Confusion Matrix : 
 [[418 138]
 [219 337]]
Accuracy on test set:    0.6789568345323741
Sensitivity:    0.6061151079136691 	 Specificity:    0.7517985611510791
Precision  : 0.7094736842105264
Area Under Curve:    0.6789568345323741
(1112, 1024) (1112,)
(4984, 1024) (730, 1024)
(207907, 1024) (23200, 1024)
Training Dataset
(9968, 1024) (9968,)
Vali

(5112, 1024) (552, 1024)
(207617, 1024) (22065, 1024)
Training Dataset
(10224, 1024) (10224,)
Validation Dataset




(1104, 1024) (1104,)
Independent Testing Matthews Correlation:  0.32315831568146847
Confusion Matrix : 
 [[417 135]
 [242 310]]
Accuracy on test set:    0.6585144927536232
Sensitivity:    0.5615942028985508 	 Specificity:    0.7554347826086957
Precision  : 0.6966292134831461
Area Under Curve:    0.6585144927536233
(1104, 1024) (1104,)
(5199, 1024) (465, 1024)
(208717, 1024) (20965, 1024)
Training Dataset
(10398, 1024) (10398,)
Validation Dataset




(930, 1024) (930,)
Independent Testing Matthews Correlation:  0.3576066538192496
Confusion Matrix : 
 [[360 105]
 [197 268]]
Accuracy on test set:    0.6752688172043011
Sensitivity:    0.5763440860215053 	 Specificity:    0.7741935483870968
Precision  : 0.7184986595174263
Area Under Curve:    0.675268817204301
(930, 1024) (930,)
(5103, 1024) (561, 1024)
(205551, 1024) (24131, 1024)
Training Dataset
(10206, 1024) (10206,)
Vali